# This notebook will demonstrate the the capability to act as a SQL agent

## Connecting to the SQL database (here: products.db)

In [8]:
import os
from mypackage import finder
from langchain_community.utilities import SQLDatabase

data_dir = os.path.join(finder.get_git_root(), "data")
db_name = 'products.db'
db_path = os.path.join(data_dir, db_name)

# Setting up database handler
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
# print(f'Sample output: {db.run(f"SELECT * FROM {db.get_usable_table_names()[0]} LIMIT 3;")}')

Dialect: sqlite
Available tables: ['products']


## Setting up the chat model

In [9]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "llama3.2",
    model_provider="ollama",
    verbose=True,
    stop=["\n\n"],
    temperature=0.7,
    max_tokens=1024,
)

## Setting up the SQL database toolkit

In [10]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool name: {tool.name}, description: {tool.description}")

Tool name: sql_db_query, description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
Tool name: sql_db_schema, description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
Tool name: sql_db_list_tables, description: Input is an empty string, output is a comma-separated list of tables in the database.
Tool name: sql_db_query_checker, description: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!


## Setting up the sytem prompt

In [11]:
system_prompt = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=3,
)

## Setting up the agent

In [12]:
from langchain.agents import create_agent
agent = create_agent(
    model,
    tools,
    system_prompt=system_prompt
)

## Running the agent

In [13]:
from mypackage import userinput

user_query = userinput.get_user_input("Stelle deine Frage!", default="Welche Informationen enthält die Datenbank?")

for step in agent.stream(
    {"messages": [{"role": "user", "content": user_query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Welche Informationen enthält die Datenbank?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (d7ae9de1-7e4f-48b4-8031-77f1c974ca71)
 Call ID: d7ae9de1-7e4f-48b4-8031-77f1c974ca71
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

products
================================== Ai Message ==================================

Die Datenbank enthält eine Tabelle namens "products". Ich werde nun die Spalten dieser Tabelle auflisten, um zu sehen, welche Informationen verfügbar sind.
